# Cert-Elastic on Google Colab

このノートブックは、Google Drive にプロジェクトを配置して Colab 上で VS Code と同等の手順で実行できるようにしたものです。GPU を有効化してから実行してください。

- 実行対象: `run_evaluate_cert.py`（ログ計測＋要約）
- 保存先: Google Drive (`/content/drive/MyDrive/Cert-Elastic`) 配下に成果物を永続化
- モデル: 省メモリのためデフォルトで 4bit 量子化を推奨します（Colab T4 / L4 でも動作）

In [ ]:
# This file will be replaced by the notebook editor

## 1) 必要ライブラリとGPUの確認
以下で Python/Pip バージョンと GPU を確認します。

In [ ]:
!python -V
!pip -V
!nvidia-smi || echo "No NVIDIA GPU"

import torch, platform
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("python:", platform.python_version())

In [ ]:
# Preflight: GPU/torch/transformers の互換チェックとフォールバック
import sys, subprocess, json
import torch
import transformers

print('preflight: torch', torch.__version__, 'cuda?', torch.cuda.is_available())
print('preflight: transformers', transformers.__version__)

# SDPAの可用性チェック（bf16 + A100 前提）
need_reinstall = False
try:
    sdpa_ok = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
    print('preflight: sdpa available =', sdpa_ok)
except Exception:
    sdpa_ok = False

if not torch.cuda.is_available():
    print('[preflight] CUDA が無効。torch の再インストールを試みます。')
    need_reinstall = True

if need_reinstall:
    # フォールバック: デフォルト index から最新安定版を導入
    # （このセルを再実行するときはランタイム再起動が必要な場合があります）
    print('[preflight] Reinstalling torch from default index...')
    code = subprocess.call([sys.executable, '-m', 'pip', 'install', '-U', 'torch', 'torchvision', 'torchaudio'])
    print('[preflight] pip exit code =', code)
    import importlib
    importlib.invalidate_caches()
    import torch as _t
    print('after reinstall: torch', _t.__version__, 'cuda?', _t.cuda.is_available())

## 2) Google Drive のマウント
Drive をマウントして、プロジェクトのルートパスを設定します。

In [ ]:
from google.colab import drive
import os, pathlib

drive.mount('/content/drive')
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_NAME = 'Cert-Elastic'
PROJECT_DIR = f'{DRIVE_ROOT}/{PROJECT_NAME}'
print('PROJECT_DIR =', PROJECT_DIR)

# HF cache to Drive to persist models between sessions
os.environ['HF_HOME'] = f'{DRIVE_ROOT}/.cache/huggingface'
os.environ['HF_DATASETS_CACHE'] = f'{DRIVE_ROOT}/.cache/huggingface/datasets'
os.environ['TRANSFORMERS_CACHE'] = f'{DRIVE_ROOT}/.cache/huggingface/hub'
pathlib.Path(os.environ['HF_HOME']).mkdir(parents=True, exist_ok=True)

## 3) Drive 内にプロジェクト用フォルダを作成
既存ならスキップされます。出力やログの保存先も合わせて作成します。

In [ ]:
import os
from datetime import datetime

SUBDIRS = ['src', 'data', 'outputs', 'models', 'logs', 'runs']
for sd in SUBDIRS:
    os.makedirs(f'{PROJECT_DIR}/{sd}', exist_ok=True)
print('created:', [f'{PROJECT_DIR}/{sd}' for sd in SUBDIRS])

## 4) リポジトリの取得 or 手元のZIP配置
- 公開GitHub: git clone（下の例ではダミーURLなので必要に応じて変更）
- 既に PC から Drive に `Cert-Elastic` フォルダをコピー済みならこのセルはスキップしてOK

In [ ]:
# いずれかを利用（既に Drive にコピー済みなら不要）
# !git clone --depth 1 https://github.com/yourname/Cert-Elastic.git /content/Cert-Elastic
# !unzip -o /content/Cert-Elastic.zip -d /content/
# !rsync -a /content/Cert-Elastic/ "$PROJECT_DIR/"

## 5) 依存関係のインストール
`requirements.txt` と PyTorch をインストールします。Colab GPU は Linux なので `bitsandbytes` も利用可能です。

In [ ]:
%cd "$PROJECT_DIR"
!python -m pip install -U pip wheel setuptools
# pip / setuptools 更新直後に IPython の依存警告が出る場合があるため補修
!pip install -U "ipython>=8.12" "jedi>=0.16"
# Colab の CUDA に合う公式 index から torch を導入（バージョンは適宜）
!pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio

# プロジェクトの依存
!pip install -r requirements.txt

# 追加: ベンチ用に lm-eval（EleutherAI/lm-evaluation-harness）や便利ツールも導入
!pip install -U lm-eval einops tiktoken

import torch
print('torch:', torch.__version__, 'cuda available:', torch.cuda.is_available())

## 6) システム依存のインストール（必要時）
このプロジェクトでは通常不要です。必要になった場合のみ以下を利用してください。

In [ ]:
# 例:
# !apt-get update -y && apt-get install -y libgl1 libglib2.0-0 graphviz

## 7) PYTHONPATH と作業ディレクトリの設定
ノートブックからローカルパッケージ `cert_elastic` を import 可能にします。

In [ ]:
%cd "$PROJECT_DIR"
import sys, os
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
print(sys.path[:3])

# import テスト
try:
    import cert_elastic
    print('cert_elastic imported:', cert_elastic.__file__)
except Exception as e:
    print('import error:', e)

## 8) 環境変数・.env の設定（任意）
Hugging Face Hub のトークンなどが必要な場合に設定します。

In [ ]:
import os
# 例: os.environ['HF_TOKEN'] = 'hf_XXXXXXXXXXXXXXXX'
# from huggingface_hub import login
# login(token=os.environ['HF_TOKEN'])

## 9) データセットのダウンロード/配置（任意）
本プロジェクトではサンプルプロンプトのみなので省略可。外部データが必要になったらここで準備してください。

## 10) テスト（任意）
テストがある場合はここで実行します。現状このリポジトリにはテストはありません。

## 11) アプリ/スクリプトの実行
`run_evaluate_cert.py` を Colab 向けの低メモリ設定で実行します。

In [ ]:
%cd "$PROJECT_DIR"

# A100向けデフォルト（高性能設定）
MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'bfloat16'        # A100ならbf16推奨
LOAD_IN_4BIT = 0          # 量子化は不要
ATNN_IMPL = 'eager'        # 注意: ロギングでは attentions が必要なため eager を使用
MAX_NEW_TOKENS = 256
EVAL_PROMPTS_N = 50
RUN_FAST = 1              # 簡易速度計測をON

!python run_evaluate_cert.py \
  --model_id {MODEL_ID} \
  --dtype {DTYPE} \
  --load_in_4bit {LOAD_IN_4BIT} \
  --device_map auto \
  --attn_impl {ATNN_IMPL} \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --eval_prompts_n {EVAL_PROMPTS_N} \
  --run_fast {RUN_FAST} \
  --out_dir runs

## 12) 実行結果の保存とDriveへの永続化
`runs/` 以下に生成された最新の run ディレクトリを参照し、主要ファイルを表示します。

In [ ]:
import json, glob, os
from pathlib import Path

%cd "$PROJECT_DIR"
run_dirs = sorted(glob.glob('runs/run_*'))
print('runs found:', run_dirs[-3:])
if run_dirs:
    latest = run_dirs[-1]
    print('latest run:', latest)
    for f in ['config.eval.json','results.logging.json','summary.json','cert_checks.csv']:
        p = Path(latest)/f
        if p.exists():
            print(f'--- {p} (head) ---')
            if p.suffix == '.json':
                print(json.dumps(json.loads(p.read_text()) if p.read_text() else {}, ensure_ascii=False)[:1000])
            else:
                print('\n'.join(p.read_text().splitlines()[:10]))
        else:
            print(f'{p} not found')

## 13) セッション再開用の短縮セル
再接続時は以下のみ実行すればすぐ再開できます。

In [ ]:
from google.colab import drive
import os, sys

drive.mount('/content/drive', force_remount=False)
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_NAME = 'Cert-Elastic'
PROJECT_DIR = f'{DRIVE_ROOT}/{PROJECT_NAME}'
%cd "$PROJECT_DIR"
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# ここからすぐ 11) を実行可能

## 14) lm-eval-harness で複数データセット評価
A100推奨。HumanEval/MBPP はコード実行を伴うため安全確認を自動で許可しています（ノート上で完結）。
- 対応エイリアス: gsm8k, humaneval, mbpp, hendrycks_math
- time削減のため limit を小さめに調整可能（厳密評価なら外す）

In [ ]:
%cd "$PROJECT_DIR"

MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'bfloat16'   # A100向け
ATNN_IMPL = 'sdpa'
EPSILON = 0.02
ALPHA = 0.5
BETA = 2.0
TASKS = 'gsm8k,humaneval,mbpp,hendrycks_math'
FEWSHOT = 0
LIMIT = 0   # 0 or None で全件（A100で本評価）。時間短縮は 100 など。

!python -m bench.run_lmeval_cert \
  --model_id {MODEL_ID} \
  --dtype {DTYPE} \
  --attn_impl {ATNN_IMPL} \
  --epsilon {EPSILON} \
  --alpha {ALPHA} \
  --beta {BETA} \
  --tasks {TASKS} \
  --fewshot {FEWSHOT} \
  --limit {LIMIT} \
  --out_dir runs

## 15) 論文風スクリプトで Throughput/Acc をまとめて計測
手軽に GSM8K / MATH / HumanEval / MBPP を擬似的に一気に回し、`runs/results.certelastic.paperstyle.json` を作ります。

In [ ]:
%cd "$PROJECT_DIR"

MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'bfloat16'
ATNN_IMPL = 'sdpa'
EPSILON = 0.02
ALPHA = 0.5
BETA = 2.0
TASKS = 'gsm8k,math,humaneval,mbpp'
N_ITEMS = 512     # 本評価
GEN_LEN = 512
TEMP = 0.0
TOP_P = 1.0
TOP_K = 0

!python -m bench.run_certelastic_paperstyle \
  --model_id {MODEL_ID} \
  --dtype {DTYPE} \
  --attn_impl {ATNN_IMPL} \
  --epsilon {EPSILON} \
  --alpha {ALPHA} \
  --beta {BETA} \
  --tasks {TASKS} \
  --n_items {N_ITEMS} \
  --gen_len {GEN_LEN} \
  --temperature {TEMP} \
  --top_p {TOP_P} \
  --top_k {TOP_K} \
  --out runs/results.certelastic.paperstyle.json

## 16) 表・図の生成（オプション）
`configs/paper_tables.template.yaml` と、上で作成された `runs/results.certelastic.paperstyle.json` を組み合わせて図表を作ります。

In [ ]:
%cd "$PROJECT_DIR"

PAPER_YAML = 'configs/paper_tables.template.yaml'
OURS_JSON = 'runs/results.certelastic.paperstyle.json'
OUT_DIR = 'runs/figs'

!python viz/make_tables_and_plots.py \
  --paper_template {PAPER_YAML} \
  --our_results {OURS_JSON} \
  --out_dir {OUT_DIR}

import os
print('figures saved to', os.path.abspath(OUT_DIR))

## 17) 大規模データセットを事前ダウンロード（キャッシュ永続化）
長時間評価の前に `datasets` をローカルキャッシュ（Drive）へプリフェッチしておくと、実行が安定・高速化します。

In [ ]:
import os
# HuggingFace datasets を事前ダウンロード
from datasets import load_dataset

# GSM8K（train/test）
_ = load_dataset('gsm8k', 'main')
# Hendrycks MATH（全カテゴリをプリフェッチ）
for _cfg in ['algebra','counting_and_probability','geometry','intermediate_algebra','number_theory','prealgebra','precalculus']:
    try:
        _ = load_dataset('hendrycks/competition_math', _cfg)
    except Exception:
        try:
            _ = load_dataset('competition_math', _cfg)
        except Exception as e:
            print('[warn] MATH prefetch failed for', _cfg, ':', e)
# HumanEval
_ = load_dataset('openai_humaneval')
# MBPP（sanitized）
_ = load_dataset('mbpp', 'sanitized')

print('datasets cached under:', os.environ.get('HF_DATASETS_CACHE', '~/.cache/huggingface/datasets'))

## 18) クイック・サニティチェック（5分以内）
A100 課金前に、パイプライン全体が動くか最小構成で検証します。モデル/依存/データセット/Cert-Elastic の一通りを通します。

以下の3ステップを順に実行してください（5分以内の煙テスト）。
- 基礎評価を最小設定で実行
- lm-eval を limit=5 で極小実行（gsm8k, hendrycks_math）
- 論文風まとめを極小実行（n_items=8, gen_len=64）

In [ ]:
%cd "$PROJECT_DIR"

# 1) 最小の基礎評価（短縮）
!python run_evaluate_cert.py \
  --model_id mistralai/Mistral-7B-Instruct-v0.3 \
  --dtype bfloat16 \
  --load_in_4bit 0 \
  --device_map auto \
  --attn_impl eager \
  --max_new_tokens 32 \
  --eval_prompts_n 3 \
  --run_fast 1 \
  --out_dir runs

# 2) lm-eval を極小サイズで煙テスト（こちらは sdpa のままでOK）
!python -m bench.run_lmeval_cert \
  --model_id mistralai/Mistral-7B-Instruct-v0.3 \
  --dtype bfloat16 \
  --attn_impl sdpa \
  --epsilon 0.02 \
  --alpha 0.5 \
  --beta 2.0 \
  --tasks gsm8k,hendrycks_math \
  --fewshot 0 \
  --limit 5 \
  --out_dir runs

# 3) 論文風まとめの極小版（こちらも sdpa でOK）
!python -m bench.run_certelastic_paperstyle \
  --model_id mistralai/Mistral-7B-Instruct-v0.3 \
  --dtype bfloat16 \
  --attn_impl sdpa \
  --epsilon 0.02 \
  --alpha 0.5 \
  --beta 2.0 \
  --tasks gsm8k,math \
  --n_items 8 \
  --gen_len 64 \
  --temperature 0.0 \
  --top_p 1.0 \
  --top_k 0 \
  --out runs/results.certelastic.paperstyle.json

print('Quick sanity done. Check latest runs/ directory.')

### 18.1) サニティ結果のざっと確認
最新の `runs/run_*` から主要ファイルの存在と先頭をチェックして、パイプラインの通過を確認します。

In [ ]:
%cd "$PROJECT_DIR"
import json, glob
from pathlib import Path

run_dirs = sorted(glob.glob('runs/run_*'))
print('runs found:', run_dirs[-3:])
if run_dirs:
    latest = run_dirs[-1]
    print('latest:', latest)
    for f in ['config.eval.json','results.logging.json','summary.json','cert_checks.csv','lmeval_baseline.json','lmeval_cert.json','results.certelastic.paperstyle.json']:
        p = Path(latest)/f
        if p.exists():
            print(f'--- {p} (head) ---')
            if p.suffix == '.json':
                try:
                    print(json.dumps(json.loads(p.read_text()) if p.read_text() else {}, ensure_ascii=False)[:800])
                except Exception:
                    print(p.read_text()[:800])
            else:
                print('\n'.join(p.read_text().splitlines()[:10]))
        else:
            print(f'{p} not found')

## 19) 全セル順実行（夜間バッチ用）
ボタン一つで上から順に主要セルを実行します。Colab のセッション切れを考慮して、適宜 Drive キャッシュを活用します。

In [ ]:
# Colab には notebook 内セルを順次トリガーする公式APIがないため、
# 主要処理を関数化して順に呼ぶ“擬似一括実行”を用意します。
# ここでは、依存インストール済み前提で、(17)->(14)->(15)->(16) の順を走らせます。

%cd "$PROJECT_DIR"

import os, json, sys, subprocess

steps = [
    [sys.executable, '-c', 'from datasets import load_dataset;\n'
     "load_dataset('gsm8k','main'); load_dataset('hendrycks/competition_math'); "
     "load_dataset('openai_humaneval'); load_dataset('mbpp','sanitized'); print('prefetched')"],
    [sys.executable, '-m', 'bench.run_lmeval_cert',
     '--model_id','mistralai/Mistral-7B-Instruct-v0.3',
     '--dtype','bfloat16','--attn_impl','sdpa',
     '--epsilon','0.02','--alpha','0.5','--beta','2.0',
     '--tasks','gsm8k,humaneval,mbpp,hendrycks_math',
     '--fewshot','0','--limit','0','--out_dir','runs'],
    [sys.executable, '-m', 'bench.run_certelastic_paperstyle',
     '--model_id','mistralai/Mistral-7B-Instruct-v0.3',
     '--dtype','bfloat16','--attn_impl','sdpa',
     '--epsilon','0.02','--alpha','0.5','--beta','2.0',
     '--tasks','gsm8k,math,humaneval,mbpp',
     '--n_items','512','--gen_len','512',
     '--temperature','0.0','--top_p','1.0','--top_k','0',
     '--out','runs/results.certelastic.paperstyle.json'],
    [sys.executable, 'viz/make_tables_and_plots.py',
     '--paper_template','configs/paper_tables.template.yaml',
     '--our_results','runs/results.certelastic.paperstyle.json',
     '--out_dir','runs/figs']
]

for i, cmd in enumerate(steps, 1):
    print(f'\n[batch] step {i}/{len(steps)} ->', ' '.join(cmd[:6]), '...')
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f'[batch] step {i} failed:', e)
        break
else:
    print('[batch] all steps completed')